<img alt="Pascual logo" height="120px" src="https://github.com/andresperez86/Data_Analysis_20252/blob/master/cropped-Institucion_Pascual_Bravo_Logo.png?raw=true" align="center" hspace="10px" vspace="10px" style="width:520px;height:152px;">
<h1><font color='01b3c2'> <center>  MapReduce.</font> </center>

<font  face="Courier New" size="3">
 <p><center>Prof. Andres Fernando Pérez  MSc.</center></p>
</font>



## 1. Introduction. <br>
<p align="justify"> <font face="Verdana" size="2.5">
MapReduce is a programming model designed to process <em> large data sets efficiently. It relies on parallel and distributed computation </em>, making it fault-tolerant and highly scalable.<br>
This model has been implemented in multiple platforms: Google’s internal system, the open-source framework Hadoop, and more recently, Apache Spark, which extends MapReduce by allowing in-memory computation.<br>
To execute a MapReduce process, programmers only need to define two functions:<em> Map</em> and <em>Reduce</em>. The system itself takes care of parallelization, task scheduling, and fault recovery.

MapReduce provides several benefits for big data analysis:

<ul>
 <li>   <b>Scalability:</b> Efficiently handles petabytes of data in Distributed File Systems (DFS). </li>
 <li> <b>Flexibility:</b>Works with diverse data types and sources.</li>
 <li><b>Speed:</b> Parallel processing reduces execution time.</li>
 <li><b>Simple:</b>Code can be written in languages like Java, Python, R, C++, or SQL.</li>
 </ul>    

<center><img src="https://github.com/andresperez86/Data_Analysis_20252/blob/master/map-reduce.png?raw=true"   height="330" width="530"></center>
<p><caption><center><font color='0B5345'> <b>Figure 1: </b><br> A MapReduce process.</font></center></caption></p>
Figure 1 depicts the operational scenario:
<ul align="justify"> <font face="Verdana" size="2.5">
<li><b>Input: </b>Some number of Map tasks  each are given one or more partitions or file fragments, generally  taken from a distributed file system. </li>
<li><b> Map phase: </b> These Map tasks turn data into the file fragments into a sequence of <em>key-value</em> pairs. The way key-value pairs are produced from the input data is determined by the code written by the programmer of the Map function. Keys are not “keys” in the usual sense; they do not have to be unique.</li>
<li><b>Group By key:</b> The key-value pairs from each Map task are collected by a master controller and sorted by key. The keys are divided among all the Reduce tasks, so all key-value pairs with the same key wind up at the same Reduce task.</li>
<li><b>Reduce Phase: </b> The Reduce tasks work on one key at a time, and combine all the values associated with that key in some way. The manner of combination of values is determined by the code written by the programmer of the Reduce function. In the original versión of map-reduce, the output of this phase is written to disk, but new extensions such as SPARK allow it to hold data in the main memory.</li>
</ul>
</font>
</p>


## 2. Example.
<p align="justify"><font face="Courier New" size="2.5"><b> <font color='01b3c2'> Goal:</font></b> Calculate the frequency of each word per document in a large collection of documents.<br><br> <center><img src="https://github.com/andresperez86/Data_Analysis_20252/blob/master/map-reduce-1.png?raw=true" height="550" width="800"></center> <caption><center><font color='0B5345'> <b>Figure 2: </b><br> A MapReduce process for word frequency per document.</font></center></caption> <ul><font face="Verdana" size="2.5"> <li>The <em>user program</em> starts a <em>master controller process</em> and a set of <em>worker processes</em> running across different compute nodes. Typically, a worker is responsible for either <em>map tasks</em> (<em>Map worker</em>) or <em>reduce tasks</em> (<em>Reduce worker</em>), but not both simultaneously.</li> <li>The <em>master controller</em> plays a central role in managing the execution:</li> <ul align="justify"><font face="Verdana" size="2.5"> <li>It creates a number of <em>map tasks</em> and <em>reduce tasks</em>, as specified by the user program. Normally, one Map task is created for each input fragment, while the number of Reduce tasks is often smaller.</li> <li>It tracks the status of every task (idle, running, or completed). Once a worker finishes a task, it reports back to the master, which then assigns the worker a new task.</li> </ul> <li>Each <em>map task</em> processes one or more fragments of the input file(s). A fragment is a self-contained subset of records (e.g., lines, tuples, or full documents), ensuring no record is split across fragments.</li> <ul align="justify"><font face="Verdana" size="2.5"> <li>In this example, the input is a set of documents represented as $ \{ doc_{id}, \{ w_j\} \}$. Each Map task reads a document, breaks it into words $w_1,w_2, \dots ,w_n$, and produces key-value pairs of the form $\{(key,value)\} = \{(w_j, doc_{id})\}$.

If a word $w_j$ appears $m$ times in a document, the Map task generates $m$ identical pairs $(w_j, doc_{id})$.</li>

</ul> <li>All key-value pairs are automatically grouped by key using a hash-based partitioning. For each word, the system builds a list of document IDs where it occurs. This intermediate data is stored in local files, and the <em>master</em> keeps track of their locations for subsequent Reduce tasks.</li> <li>The master assigns <em>reduce tasks</em> to workers and provides them with the corresponding intermediate files. For example: <ul> <li>Worker–5 receives tuples with keys $(w_1, w_2, w_3)$.</li> <li>Worker–6 receives tuples with keys $(w_4, w_5, w_6)$.</li> </ul></li> <li>Each <em>reduce task</em> executes the user-defined Reduce function. It aggregates the values for each key (word) and produces the final output. For instance: if the word $w_5$ appears three times in document $id = 6$, the reducer outputs: $$(w_5, \{ (doc_6, 3) \})$$</li> <li><b>MapReduce with Combiner.</b> When the Reduce function is both associative and commutative, a <em>combiner</em> can be introduced to partially aggregate data during the Map phase. This reduces network traffic.</li> <ul align="justify"><font face="Verdana" size="2.5"> <li>In our example, instead of emitting one pair per occurrence, the combiner summarizes counts locally, producing pairs of the form: $$ (w_j, [doc_{id}, frequency]) $$ Here, <em>frequency</em> is the number of times word $w_j$ appears in that document fragment.

This optimization reduces the amount of intermediate data sent across the network. However, final grouping and aggregation are still required at the Reduce stage, since each Map task may produce a partial result for the same word.</li>

</ul> </ul> </font></p>



## Comparison: With vs Without Combiner
<p align="center"> <font face="Verdana" size="2.5"> <table border="1" cellpadding="6" cellspacing="0" style="border-collapse:collapse; text-align:center;"> <tr bgcolor="#01b3c2" style="color:white;"> <th><b>Step</b></th> <th><b>Without Combiner</b></th> <th><b>With Combiner</b></th> </tr> <tr> <td><b>Map Output</b></td> <td>Each word occurrence generates a key-value pair. <br> Example (doc 6 contains “w5” three times): <br> $(w_5, doc_6)$, $(w_5, doc_6)$, $(w_5, doc_6)$</td> <td>Map task aggregates counts locally. <br> Example (doc 6 contains “w5” three times): <br> $(w_5, [doc_6, 3])$</td> </tr> <tr> <td><b>Intermediate Data Size</b></td> <td>Large (one entry per word occurrence). <br> Higher network traffic.</td> <td>Reduced (one entry per word per document). <br> Less network traffic.</td> </tr> <tr> <td><b>Network Transmission</b></td> <td>All pairs are shuffled to Reducers. <br> Costly in large datasets.</td> <td>Only aggregated pairs are shuffled. <br> More efficient use of bandwidth.</td> </tr> <tr> <td><b>Reducer Input</b></td> <td>Many duplicates to process. <br> Example: 3 entries for “w5” in doc 6.</td> <td>Fewer, aggregated entries. <br> Example: 1 entry for “w5” in doc 6 with count = 3.</td> </tr> <tr> <td><b>Final Output</b></td> <td>$(w_5, \{(doc_6, 3)\})$ <br> (same result, but more overhead).</td> <td>$(w_5, \{(doc_6, 3)\})$ <br> (same result, less overhead).</td> </tr> </table> </font> </p>

## 3. Failure management.
<p align="justify"><font face="Verdana" size="2.5"> In large-scale, parallel, and distributed systems like MapReduce, failures are expected due to the complexity and number of nodes involved. The framework is designed to be **fault-tolerant**, so jobs can still complete successfully even if some components fail. Several possible scenarios are: <ul align="justify"> <li><em>Master controller fails:</em> If the Master process fails, the entire MapReduce job must be restarted. The Master is a single point of coordination, and without it, no recovery is possible. In contrast, all other types of failures can be detected and managed by the Master, allowing the job to eventually finish.</li> <li><em>A Map worker fails:</em> The Master periodically sends "heartbeat" signals (pings) to all workers. If a Map worker does not respond, the Master marks it as failed. Any Map tasks assigned to that worker are rescheduled and executed on another available worker.</li> <li><em>Compute node with Map workers fails:</em> If an entire node crashes (where one or several Map workers reside), the Master also detects the failure. In this case, **all Map tasks from that node must be re-executed**, even if they had already completed. The reason is that the intermediate output (key-value pairs) stored on that node is no longer accessible to Reduce workers. The Master must then notify each Reduce task about the new location of their required input.</li> <li><em>A Reduce worker fails:</em> Failures in Reduce workers are easier to handle. If a Reduce task fails, the Master resets its status to *idle* and reschedules it on another available worker. Since input data for Reduce tasks is replicated or can be regenerated from Map outputs, recovery is straightforward.</li> <li><em>Distributed File System (DFS) reliability:</em> MapReduce relies on a distributed file system (such as HDFS in Hadoop) that replicates each file fragment across multiple nodes. This replication ensures that data remains available and recoverable even if a disk or a node fails, further improving system fault tolerance.</li> </ul> </font></p>


## Bibliography and related resources.
<p align="justify"><font face="Verdana" size="2.5"> [1]&nbsp;&nbsp;&nbsp;&nbsp; J. Leskovec, A. Rajaraman, y J. D. Ullman, <cite>Mining of Massive Datasets</cite>, 2nd ed. New York, NY, USA: Cambridge University Press, 2014. <a href="http://www.mmds.org/"> here</a> </p>

<p align="justify"><font face="Verdana" size="2.5"> [2]&nbsp;&nbsp;&nbsp;&nbsp;Dean, J. and Ghemawat, S. 2004. <cite>MapReduce: Simplified data processing on large clusters. </cite>In Proceedings of Operating Systems Design and <br> &emsp; &emsp; Implementation (OSDI). San Francisco, CA. 137-150. <a href="https://static.googleusercontent.com/media/research.google.com/es//archive/mapreduce-osdi04.pdf"> here</a></p>

<p align="left"><b><font face='Courier New' color="white" align="left" size=4>Copyright.</font></b>
<img alt="GIIAM" height="120px" src="https://pascualbravo.edu.co/investigacion/giiam/" align="right" hspace="10px" vspace="0px" height="120" width="350"">
                                                                                                                              
<font face='Verdana' size="2.5">
Andres Fernando Perez G. <a href="https://scienti.minciencias.gov.co/cvlac/visualizador/generarCurriculoCv.do?cod_rh=0000347507">  CvLAC</a><br>
I.U Pascual Bravo.<br>
Calle 73 # 73A – 226<br>
Medellín, Colombia. South America.
    
</p>
</font>
    
</p>
</font>

<center><b><font color='01b3c2' face="Lucida Calligraphy,Comic Sans MS,Lucida Console" size="4">I.U Pascual Bravo.</font></b> </center>